## 諸々インポート

In [98]:
import os
import chainer
from chainer import Chain
from chainer import Variable
from chainer import training, report
from chainer import optimizers, iterators
from chainer.training import extensions
from chainer import cuda
import chainer.functions as F
import chainer.links as L

import matplotlib.pyplot as plt

from sklearn.datasets import fetch_mldata
import numpy as np
import pandas as pd

import pickle
from PIL import Image

cuda.get_device_from_id(0).use()
xp = cuda.cupy

## Generatorのクラス

In [99]:
class Generator(Chain):
    def __init__(self, z_dim):
        super(Generator, self).__init__(
            l1=L.Linear(z_dim, 3 * 3 * 512),
            
            dc1=L.Deconvolution2D(512, 256, 2, stride=2, pad=1, ),
            dc2=L.Deconvolution2D(256, 128, 2, stride=2, pad=1, ),
            dc3=L.Deconvolution2D(128, 64, 2, stride=2, pad=1, ),
            dc4=L.Deconvolution2D(64, 1, 3, stride=3, pad=1,),
            
            bn1=L.BatchNormalization(512),
            bn2=L.BatchNormalization(256),
            bn3=L.BatchNormalization(128),
            bn4=L.BatchNormalization(64),
        )
        self.z_dim = z_dim
        
    def __call__(self, z):
        h = self.l1(z)
        h = F.reshape(h, (z.shape[0], 512, 3, 3))
        
        h1 = F.relu(self.bn1(h))
        h2 = F.relu(self.bn2(self.dc1(h1)))
        h3 = F.relu(self.bn3(self.dc2(h2)))
        h4 = F.relu(self.bn4(self.dc3(h3)))
        x = F.sigmoid(self.dc4(h4))
        return x


## Discriminatorのクラス

In [100]:
class Discriminator(Chain):
    def __init__(self, ):
        super(Discriminator, self).__init__(
            c1=L.Convolution2D(1, 64, 3, stride=3, pad=1, ),
            c2=L.Convolution2D(64, 128, 2, stride=2, pad=1, ),
            c3=L.Convolution2D(128, 256, 2, stride=2, pad=1, ),
            c4=L.Convolution2D(256, 512, 2, stride=2, pad=1, ),
            
            l1=L.Linear(3 * 3 * 512, 2),
            
            bn1=L.BatchNormalization(128),
            bn2=L.BatchNormalization(256),
            bn3=L.BatchNormalization(512),
        )
    
    def __call__(self, x):
        h1 = F.relu(self.c1(x))
        h2 = F.relu(self.bn1(self.c2(h1)))
        h3 = F.relu(self.bn2(self.c3(h2)))
        h4 = F.relu(self.bn3(self.c4(h3)))
        y = self.l1(h4)
        return y


## Updaterのクラス

In [101]:
class GANUpdater(training.StandardUpdater):
    def __init__(self, *args, **kwargs):
        self.gen, self.dis = kwargs.pop('models')
        super(GANUpdater, self).__init__(*args, **kwargs)
        
    def dis_loss(self, y_fake, y_real):
        batch_size = len(y_fake)
        real_loss = F.sum(F.softplus(-y_real)) / batch_size
        fake_loss = F.sum(F.softplus(y_fake)) / batch_size
        loss = real_loss + fake_loss
        report({'loss': loss}, self.dis)
        return loss
    
    def gen_loss(self, y_fake):
        batch_size = len(y_fake)
        loss = F.sum(F.softplus(-y_fake)) / batch_size
        report({'loss': loss}, self.gen)
        return loss
    
    def update_core(self):
        gen_optimizer = self.get_optimizer('gen')
        dis_optimizer = self.get_optimizer('dis')
        
        batch = self.get_iterator('main').next()
        x_real = Variable(self.converter(batch, self.device))
        batch_size = len(x_real)
        
        y_real = dis(x_real)
        
        z = xp.random.uniform(-1, 1, (batch_size, self.gen.z_dim))
        z = z.astype(dtype=xp.float32)
        x_fake = gen(z)
        y_fake = dis(x_fake)
        
        dis_optimizer.update(self.dis_loss, y_fake, y_real)
        gen_optimizer.update(self.gen_loss, y_fake)
        

## 表示用のクラス

In [110]:
def image(gen, rows, cols, dst):
    @training.make_extension()
    def make_image(trainer):
        n_images = rows * cols
        z = xp.random.uniform(-1, 1, (rows*cols, gen.z_dim))
        z = z.astype(dtype=xp.float32)
        with chainer.using_config('train', False):
            x = gen(z)
        x = cuda.to_cpu(x.data)

        x = np.asarray(np.clip(x * 255, 0.0, 255.0), dtype=np.uint8)
        _, _, H, W = x.shape
        x = x.reshape((rows, cols, 1, H, W))
        x = x.transpose(0, 3, 1, 4, 2)
        x = x.reshape((rows * H, cols * W))
        
        preview_dir = '{}/preview'.format(dst)
        preview_path = preview_dir +\
            '/image{:0>8}.png'.format(trainer.updater.iteration)
        if not os.path.exists(preview_dir):
            os.makedirs(preview_dir)
        Image.fromarray(x).save(preview_path)
    return make_image

## 実行する

In [112]:

gen = Generator(100)
dis = Discriminator()
gen.to_gpu()
dis.to_gpu()

gen_opt = optimizers.Adam()
gen_opt.setup(gen)
dis_opt = optimizers.Adam()
dis_opt.setup(dis)

data = fetch_mldata('MNIST original')
X = data['data']
n_train = X.shape[0]
X = np.array(X, dtype=np.float32)
X /= 255.
X = X.reshape(n_train, 1, 28, 28)
train = iterators.SerialIterator(X, 100)

updater = GANUpdater(models=(gen, dis), 
                     iterator=train, 
                     optimizer={'gen': gen_opt, 'dis': dis_opt}, 
                     device=0)

trainer = training.Trainer(updater, (1000, 'epoch'), out='result')

snapshot_interval = (100, 'epoch')
display_interval = (1000, 'iteration')
trainer.extend(extensions.LogReport(trigger=display_interval))
trainer.extend(extensions.PrintReport([
    'epoch', 'iteration', 'gen/loss', 'dis/loss',
]), trigger=display_interval)

trainer.extend(image(gen, 10, 10, 'result'),trigger=snapshot_interval)

trainer.run()

epoch       iteration   gen/loss    dis/loss  
1           1000        16.5661     0.175634    
2           2000        12.879      0.0863296   
4           3000        13.1816     0.0252714   
5           4000        12.7493     0.0686163   
7           5000        12.6176     0.052469    
8           6000        12.6163     0.0324581   
10          7000        13.9916     0.0462195   
11          8000        13.6381     0.0782531   
12          9000        13.1613     0.0437378   
14          10000       16.1907     0.0670317   
15          11000       14.2653     0.0998733   
17          12000       13.4764     0.0499048   
18          13000       14.9891     0.0482959   
20          14000       14.3505     0.0554001   
21          15000       14.3668     0.0716975   
22          16000       15.145      0.0485878   
24          17000       17.4748     0.072485    
25          18000       17.8919     0.0688703   
27          19000       17.148      0.0691575   
28          20000     

225         158000      21.8602     0.00120125  
227         159000      27.5523     0.0180425   
228         160000      21.0085     0.000724959  
230         161000      28.6809     0.0571622   
231         162000      23.56       0.00664168  
232         163000      25.2247     0.00170635  
234         164000      24.8755     0.0418254   
235         165000      22.0476     0.00114547  
237         166000      28.8246     0.00763419  
238         167000      26.4022     0.0110411   
240         168000      37.6091     0.0374194   
241         169000      29.5827     0.0224025   
242         170000      23.4183     0.000899857  
244         171000      27.0853     0.000232854  
245         172000      32.4326     0.00112034  
247         173000      33.6471     0.00614139  
248         174000      29.5073     0.0103257   
250         175000      28.411      0.00747236  
251         176000      35.4677     0.0227466   
252         177000      39.0936     0.0500725   
254         17800

450         315000      25.5821     0.00458547  
451         316000      28.7082     0.000621169  
452         317000      39.3803     0.000680988  
454         318000      27.5003     7.82932e-05  
455         319000      41.1147     0.0194668   
457         320000      27.8447     0.0244395   
458         321000      28.4635     0.0054393   
460         322000      35.2076     0.0509256   
461         323000      26.9791     0.0258404   
462         324000      26.7848     0.0299675   
464         325000      28.1988     0.0530028   
465         326000      24.2984     0.0587738   
467         327000      33.4349     0.00525445  
468         328000      31.048      0.0377211   
470         329000      22.341      0.00347228  
471         330000      29.6172     0.0336097   
472         331000      31.8711     0.00981541  
474         332000      24.0112     0.00408385  
475         333000      26.6179     0.00536662  
477         334000      28.5543     0.00761282  
478         33500

675         473000      51.8656     0.00210234  
677         474000      41.6084     0.00146734  
678         475000      48.9212     0.0265705   
680         476000      30.6244     0.0446708   
681         477000      30.6517     0.0134021   
682         478000      43.4764     0.00542749  
684         479000      32.9173     0.0120441   
685         480000      31.5342     0.000885432  
687         481000      29.3712     0.00513391  
688         482000      36.0817     0.000578843  
690         483000      53.5915     0.0481421   
691         484000      29.1859     0.000534121  
692         485000      50.6289     0.0445426   
694         486000      28.5274     0.00949959  
695         487000      26.3331     0.00299722  
697         488000      29.6632     0.000181699  
698         489000      28.3569     0.0338922   
700         490000      37.0182     0.00238565  
701         491000      33.7247     0.0420057   
702         492000      32.285      0.0136497   
704         4930

900         630000      25.6329     0.00145187  
901         631000      34.2517     0.000627338  
902         632000      35.6207     0.000871813  
904         633000      29.5928     0.00142254  
905         634000      35.9482     0.00536371  
907         635000      34.5071     0.00441378  
908         636000      60.4216     0.000708412  
910         637000      43.1956     0.0415055   
911         638000      28.3414     0.00866326  
912         639000      35.1822     0.0394872   
914         640000      32.1962     0.0059312   
915         641000      36.3234     0.00872329  
917         642000      33.8589     0.0148965   
918         643000      37.6228     0.0137768   
920         644000      31.6651     0.0273433   
921         645000      28.2245     0.0400762   
922         646000      40.1925     0.00933341  
924         647000      45.7651     0.00277394  
925         648000      46.2363     0.127629    
927         649000      26.3999     0.00876786  
928         65000